### 네이버 쇼핑 키워드 인사이트

#### 참고 서비스
- 신사임당 유투브 : https://youtu.be/Z0hL9khbpIw
- 응용 서비스 : https://whereispost.com/keyword, https://itemscout.io

#### 개요
- 키워드에 대한 검색량 데이터 수집
- 검색어에 대한 상품등록수 수집
- 수집한 데이터에 대한 인사이트 도출 : 예시) 경쟁률
- 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드와 상품 찾기

In [1]:
import requests, json, time
import pandas as pd
import signaturehelper
import warnings
warnings.filterwarnings('ignore')

#### 1. 키워드에 대한 검색량 데이터 수집

In [2]:
# API Header 관련 키 값 
BASE_URL = 'https://api.naver.com'
API_KEY = '0100000000f5597e0d6670746d1610e84af933f00a4d110978f098d0742053d8b074d2bfb6'
SECRET_KEY = 'AQAAAAD1WX4NZnB0bRYQ6Er5M/AK7oHxcJs8a57u1CdkLvFo7Q=='
CUSTOMER_ID = '2488433'

In [3]:
# API Header 함수
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

In [4]:
# keyword 관련 단어들에 대한 월별 검색 데이터 수집(naver API)
def keyword_master(keyword):
    # requests.get 함수 인자 값 설정(naver API 문서 참조)
    uri = '/keywordstool'
    method = 'GET'
    relkey_params = {
        'hintKeywords':f'{keyword}',
        'showDetail':1,#related keywords, monthly query count info only(0), all details(1)
    }
    response = requests.get(BASE_URL + uri, params=relkey_params, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    
    # response로 온 json 데이터 파싱
    datas = (response.json())["keywordList"]
    
    # 10 미만 값 처리
    for data in datas:
        if data['monthlyPcQcCnt'] == '< 10':
            data['monthlyPcQcCnt'] = 0
        if data['monthlyMobileQcCnt'] == '< 10':
            data['monthlyMobileQcCnt'] = 0
    
    # DataFrame 변환
    df = pd.DataFrame(datas)
    
    return df

In [5]:
keyword = "비타민"
df = keyword_master(keyword)

In [6]:
df.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,비타민,10300,88200,13.6,331.8,0.15,0.43,15,높음
1,건강보조식품,900,2320,3.9,12.0,0.48,0.54,15,높음
2,비타민세럼,710,5190,1.3,31.0,0.14,0.63,15,높음
3,아이오페비타민앰플,10,140,0.1,0.7,0.55,0.50,14,높음
4,종합비타민,15800,101000,46.0,979.6,0.34,1.07,15,높음


In [7]:
# 상위 20개 값만 추출
df = df.head(20)

#### 2. 검색어에 대한 상품등록수 수집
- item_count : 등록된 상품수
- total_search_count : PC, Mobile 을 합친 검색량

In [8]:
# 정적데이터-html 수집용 
from bs4 import BeautifulSoup

In [9]:
def item_count(keyword):
    # 대상 페이지 설정
    url = f"https://search.shopping.naver.com/search/all?query={keyword}&cat_id=&frm=NVSHATC"
    response = requests.get(url)
    # html에서 전체 상품 수 추출 위한 BeatifulSoup 객체 생성
    dom = BeautifulSoup(response.text, "html.parser")
    # span.subFilter_num__2x0jq의 첫 번째 값이 전체 상품 수
    element = dom.select_one("span.subFilter_num__2x0jq")
    # keyword에 해당하는 상품 수가 없을 경우 count 0으로 설정
    try:
        count = int(element.text.replace(",", ""))        
    except:
        count = 0
    return count

In [10]:
# 기존 DataFrame에서 필요한 column들만 추출
result_df = df[['relKeyword', 'monthlyPcQcCnt', 'monthlyMobileQcCnt']]

In [11]:
# relKeyword에 따른 총 상품 수(item_count) 열 생성
result_df['item_count'] = result_df['relKeyword'].apply(item_count)

In [12]:
# PC, Moblie 합계(total_search_count) 열 생성
result_df['total_search_count'] = result_df['monthlyPcQcCnt'] + result_df['monthlyMobileQcCnt']

In [13]:
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count
0,비타민,10300,88200,2817089,98500
1,건강보조식품,900,2320,2410973,3220
2,비타민세럼,710,5190,51093,5900
3,아이오페비타민앰플,10,140,105,150
4,종합비타민,15800,101000,562856,116800
5,주름개선화장품,820,5250,65206,6070
6,난소화성말토덱스트린,1370,4440,825,5810
7,바르는비타민C,70,750,465,820
8,레드프로폴리스,360,3880,6988,4240
9,건기식,1890,2230,4821,4120


#### 3. 경쟁률
- 아이템수/검색수 : 높을수록 경쟁이 심한 상품 키워드

In [14]:
# 검색 대비 아이템 비율(competition_rate) 열 생성
# 해당 열은 경쟁률의 의미를 가짐
result_df['competition_rate'] = result_df['item_count']/result_df['total_search_count']

In [15]:
# 경쟁률이 낮은 순으로 아이템 정렬
result_df.sort_values(by='competition_rate', inplace=True, ignore_index=True)

In [16]:
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count,competition_rate
0,프로폴리스효능,2430,28800,2857,31230,0.091483
1,난소화성말토덱스트린,1370,4440,825,5810,0.141997
2,바르는비타민C,70,750,465,820,0.567073
3,아이오페비타민앰플,10,140,105,150,0.700000
4,비타민씨앰플,110,1250,973,1360,0.715441
5,건기식,1890,2230,4821,4120,1.170146
6,홍경천,1100,8230,14174,9330,1.519185
7,레드프로폴리스,360,3880,6988,4240,1.648113
8,종합비타민,15800,101000,562856,116800,4.818973
9,비타민세럼,710,5190,51093,5900,8.659831


#### 인사이트 1
- 고객층 선호도 파악
- 키워드 별 PC, Mobile 유입 비율
- 위 dataframe의 상위 keyword 위주로 선정 시 경쟁률에서 우위를 점할 수 있음

#### 인사이트 2
- 고객층 선호도 파악
- 키워드 별 PC, Mobile 유입 비율

In [17]:
# PC 대비 모바일 비율(competition_rate) 열 생성
result_df['mobile_per_pc'] = result_df['monthlyMobileQcCnt']/result_df['monthlyPcQcCnt']

In [18]:
result_df.sort_values(by='mobile_per_pc', inplace=True, ignore_index=True, ascending=False)

In [19]:
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count,competition_rate,mobile_per_pc
0,아이오페비타민앰플,10,140,105,150,0.700000,14.000000
1,프로폴리스효능,2430,28800,2857,31230,0.091483,11.851852
2,비타민씨앰플,110,1250,973,1360,0.715441,11.363636
3,레드프로폴리스,360,3880,6988,4240,1.648113,10.777778
4,바르는비타민C,70,750,465,820,0.567073,10.714286
5,비타민에센스,80,790,28415,870,32.660920,9.875000
6,비타민,10300,88200,2817089,98500,28.599888,8.563107
7,홍경천,1100,8230,14174,9330,1.519185,7.481818
8,비타민세럼,710,5190,51093,5900,8.659831,7.309859
9,종합비타민추천,2060,14200,562822,16260,34.613899,6.893204
